tf.train.QueueRunner主要用于启动多个线程来操作同一个队列，启动的这些线程可以通过tf.train.Coordinator来统一管理。

In [1]:
import tensorflow as tf

#### 1. 定义队列及其操作。

In [2]:
# 声明一个先入先出队列，队列中最多100个元素，类型为float实数
queue = tf.FIFOQueue(100,"float")
# 把随机数插入到队列中
enqueue_op = queue.enqueue([tf.random_normal([1])])

# 创建一个5个线程运行队列的入队操作
# 第一个参数为被操作的队列，第二个参数表示需要启动5个线程，都运行enqueue_op操作
qr = tf.train.QueueRunner(queue, [enqueue_op] * 5)

# add_queue_runner将定义过的QueueRunner加入tf计算图上指定的集合
# 若没有指定集合，则加入默认集合tf.GraphKeys.QUENE_RUNNERS
tf.train.add_queue_runner(qr)
# 定义出队操作
out_tensor = queue.dequeue()

#### 2. 启动线程。

In [3]:
with tf.Session() as sess:
    # 协同启动线程
    coord = tf.train.Coordinator()
    # tf.train.start_queue_runners会启动tf.GraphKeys.QUENE_RUNNERS
    # 集合中的所有QueueRunner
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for _ in range(5): 
        print(sess.run(out_tensor))
    # 停止所有线程
    coord.request_stop()
    coord.join(threads)

[0.1413904]
[0.9610045]
[0.47590107]
[0.92434245]
[0.31760675]
